# Articles Similarity

prerequisites

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 81kB 4.2MB/s 
     |████████████████████████████████| 2.1MB 9.3MB/s 
     |████████████████████████████████| 1.2MB 47.4MB/s 
     |████████████████████████████████| 870kB 42.4MB/s 
     |████████████████████████████████| 3.3MB 46.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-cp37-none-any.whl size=114307 sha256=7381165b219dd62c8b7be0d571b287780e6c60657e4c06920e39ab5b77d2b30d
  Stored in directory: /root/.cache/pip/wheels/26/ea/89/d0d2e013d951b6d23270aa9ca4018b82632ab7cd933c331316
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=caed3bbdc1a5323dddfd90eb42b39ec129a16b41edf9ea447661f1183fa1a22c
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sentence-transformers sacremoses


Imports

In [ ]:
import os
import time
import IPython
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import word_tokenize

from sentence_transformers import SentenceTransformer

from google.colab import drive

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score

Initialize Google Drive

In [ ]:
drive.mount(r'/content/drive/',force_remount=True) 
data_dir = os.getcwd() + '/drive/My Drive/Colab Notebooks/AdvancedML/HW3/'
data_dir

Mounted at /content/drive/


'/content/drive/My Drive/Colab Notebooks/AdvancedML/HW3/'

Display df

In [ ]:
pd.options.display.max_colwidth = 250

Dataset

In [ ]:
df = pd.read_excel(data_dir + 'full_articles_10k.xlsx')
df.head()

,title,cord_uid,abstract,body_text
0,Chapter 6 Best practices and approaches using behavior-smart thinking in 10 tourism industry scenarios,rsgdqk1x,This chapter provides ten practical case studies illustrating application of behavioural thinking in different contexts across the travel industry. The practical examples come from a wide variety of entities – from young innovative startup compan...,"Throughout the previous chapters, this book advances the idea that behavioral economics can enhance the thinking and practices in the growing travel economy. Many of the discussions in the text propose that behavior-smart thinking helps explain s..."
1,A Comparison: Prediction of Death and Infected COVID-19 Cases in Indonesia Using Time Series Smoothing and LSTM Neural Network,k58agggx,"COVID-19 is a virus causing pneumonia, also known as Corona Virus Disease. The first outbreak was found in Wuhan, China, in the province of Hubei on December 2019. The objective of this paper is to predict the death and infected COVID-19 in Indon...","COVID-19 is a virus causing pneumonia, also known as Corona Virus Disease. The first outbreak was found in Wuhan, China, in the province of Hubei on December 2019 1,2 . In Indonesia, the first outbreak was first reported on 2 March 2020 and by 26..."
2,Environmental implication of personal protection equipment in the pandemic era: LCA comparison of face masks typologies,ercubdf9,"In the present global health emergency, face masks play a key role in limiting the diffusion of the COVID-19 pandemic, by acting as physical barriers to avoid droplets and filtrate exhalations coming from infected subjects. Since the most widespr...","Due to the fast development in the domain of communication and an ongoing trend of digitization and digitalization, manufacturing enterprises are facing important challenges in today's market environments: a continuing tendency towards reduction ..."
3,The first three weeks of lockdown in England: The challenges of detecting safeguarding issues amid nursery and primary school closures due to COVID-19,3u446iuy,"With the outbreak of COVID-19 being declared a pandemic by the World Health Organisation, educators worldwide are facing a major challenge in how to adapt, become resilient, and to monitor/detect potential safeguarding issues amid nursery and sch...","The first UK confirmed death from COVID-19 was reported on 5 th March 2020 (Public Health England, 2020) . With the COVID-19 pandemic gripping the UK, the government came under increasing pressure to close nurseries and schools. A question on clo..."
4,Appendix B Climate change and global warming: Impacts on crop production,ora47nlw,Unknown,Climate change and global warming:\nImpacts on crop production\nThe Earth's climate has varied with warming and cooling periods many times throughout history. Most of these changes have been relatively slow and due to variations in the Earth's or...


Load Bert model

In [ ]:
bert_model = SentenceTransformer('bert-base-nli-mean-tokens')

### Looking for top 10 similar titles to the different queries in task id 583 using Bert 

In [ ]:
def left_align(df: pd.DataFrame):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )
    return left_aligned_df

In [ ]:
# relevant queries for task id 583
queries = ["What has been published about information sharing and inter-sectoral collaboration?",
           "What has been published about data standards and nomenclature?",
           "What has been published about governmental public health?",
           "What do we know about risk communication?",
           "What has been published about communicating with high-risk populations?",
           "What has been published to clarify community measures?",
           "What has been published about equity considerations and problems of inequity?"
           ]
encoded_queries = [np.expand_dims(bert_model.encode(query), axis=0) for query in queries]
print(f"Shape of an encoded query: {encoded_queries[0].shape}")

In [ ]:
titles = df["title"].tolist()
titles_embeddings = bert_model.encode(titles)
print(f"titles embeddings shape: {titles_embeddings.shape}")

titles embeddings shape: (10000, 768)


In [ ]:
for query_ind, encoded_query in enumerate(encoded_queries):
  df_results = pd.DataFrame(columns=['score','title'])
  dfStyler = df_results.style.set_properties(**{'text-align': 'left'})
  dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
  query_to_title_similarity = cosine_similarity(encoded_query, titles_embeddings)[0]
  sorted_inds = np.argsort(query_to_title_similarity)[::-1]
  df_results['score'] = query_to_title_similarity[sorted_inds[:10]]
  df_results['title'] = df["title"].iloc[sorted_inds[:10]].values
  print(f"\n Query: {queries[query_ind]}")
  display(left_align(df_results))


 Query: What has been published about information sharing and inter-sectoral collaboration?


,score,title
0,0.793780,Evolution and diffusion of information literacy topics
1,0.748122,"Exploring, browsing and interacting with multi-scale structures of knowledge"
2,0.746423,Characterizing research leadership on geographically weighted collaboration network
3,0.745525,Conceptual similarity and communicative need shape colexification: an experimental study
4,0.740104,The Data Governance Act and the EU's move towards facilitating data sharing
5,0.735152,"Methodology to Assess Quality, Presence, Empathy, Attitude, and Attention in Social VR: International Experiences Use Case"
6,0.729343,Using Interteaching to Promote Online Learning Outcomes
7,0.728445,Communication and role clarity inform TeleICU use: a qualitative analysis of opportunities and barriers in an established program using AACN framework
8,0.728100,Indulging interactivity: a learning management system as a facilitative boundary object
9,0.726321,Who Studies Abroad? Understanding the Impact of Intent on Participation



 Query: What has been published about data standards and nomenclature?


,score,title
0,0.708455,Analyzing knowledge entities about COVID-19 using entitymetrics
1,0.699475,ROC Analyses Based on Measuring Evidence
2,0.689564,Camouflaged Instance Segmentation: Dataset and Benchmark Suite
3,0.688449,"Gathering, processing, and interpreting information about COVID-19"
4,0.682296,The laboratory findings and different COVID-19 severities: a systematic review and meta-analysis
5,0.670368,Chilblains and COVID‐19: can recent epidemiological data shed light on the aetiological debate?
6,0.668688,Model-based evaluation of transmissibility and reinfection for the P.1 variant of the SARS-CoV-2
7,0.668061,COVID-19 and Thromboinflammation: Is There a Role for Statins?
8,0.666669,Case Identification and Contact Tracing
9,0.665878,Introduction to the Concepts of Agr-Informatics



 Query: What has been published about governmental public health?


...

,score,title
0,0.737066,Developing a risk-adaptive technology roadmap using a Bayesian network and topic modeling under deep uncertainty
1,0.714557,Psychosocial impact of the Covid-19 pandemic: Identification of most vulnerable populations.
2,0.708352,Crisis Information Management: From Technological Potential to Societal Impact
3,0.703249,Epidemics with Behavior
4,0.698455,Prevalence of anxiety in the COVID-19 pandemic: An updated meta-analysis of community-based studies
5,0.692430,The challenges of data usage for the United States’ COVID-19 response
6,0.691771,Catastrophic risks and the pricing of catastrophe equity put options
7,0.681230,Theorizing disaster communitas
8,0.680764,COVID Stress Syndrome: Clinical and Nosological Considerations
9,0.678414,Increased mortality risk associated with statins in the CORONADO study



 Query: What has been published to clarify community measures?


,score,title
0,0.763781,Conceptual similarity and communicative need shape colexification: an experimental study
1,0.737572,The author’s response
2,0.736101,Communication and role clarity inform TeleICU use: a qualitative analysis of opportunities and barriers in an established program using AACN framework
3,0.726216,Why and how to engage expert stakeholders in ontology development: insights from social and behavioural sciences
4,0.711688,"Public Health, Communities and Consent"
5,0.710924,Are we speaking the same language? Call for action to improve theory application and reporting in behaviour change research
6,0.709580,Questioning corona—a study and research path
7,0.705810,Peer review guidance: a primer for researchers
8,0.701078,Who Studies Abroad? Understanding the Impact of Intent on Participation
9,0.700115,"COVID-19, community trials, and inclusion"



 Query: What has been published about equity considerations and problems of inequity?


,score,title
0,0.758624,Barotrauma in covid - Causes and consequences
1,0.730320,On the Quantification of Model Uncertainty: A Bayesian Perspective
2,0.720552,Inequalities and Inclusion in Exercise Referral Schemes: A Mixed-Method Multi-Scheme Analysis
3,0.719840,Should policy makers trust composite indices? A commentary on the pitfalls of inappropriate indices for policy formation
4,0.719774,Communication and role clarity inform TeleICU use: a qualitative analysis of opportunities and barriers in an established program using AACN framework
5,0.717752,Introduction: Why Theory? (Mis)Understanding the Context and Rationale
6,0.713143,Intellectual Property Rights and Access in Crisis
7,0.707993,Do we really need a knowledge-based decision theory?
8,0.701436,Graded Hoare Logic and its Categorical Semantics
9,0.699531,Value and politics: introduction to the special issue


### Looking for top 10 similar abstracts to the different queries in task id 583 using Bert 

In [ ]:
abstracts = df["abstract"].tolist()
abstract_embeddings = bert_model.encode(abstracts)
print(f"abstract embeddings shape: {abstract_embeddings.shape}")

abstract embeddings shape: (10000, 768)


In [ ]:
for query_ind, encoded_query in enumerate(encoded_queries):
  df_results = pd.DataFrame(columns=['score','title'])
  dfStyler = df_results.style.set_properties(**{'text-align': 'left'})
  dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
  query_to_abstract_similarity = cosine_similarity(encoded_query, abstract_embeddings)[0]
  sorted_inds = np.argsort(query_to_abstract_similarity)[::-1]
  df_results['score'] = query_to_abstract_similarity[sorted_inds[:10]]
  df_results['title'] = df["title"].iloc[sorted_inds[:10]].values
  print(f"\n Query: {queries[query_ind]}")
  display(left_align(df_results))


 Query: What has been published about information sharing and inter-sectoral collaboration?


,score,title
0,0.643829,"Working at a remove: continuous, collective, and configurative approaches to qualitative secondary analysis"
1,0.623041,TextEssence: A Tool for Interactive Analysis of Semantic Shifts Between Corpora
2,0.612935,Linguistic Analysis of Internet Media and Social Network Data in the Problems of Social Transformation Assessment
3,0.609051,Facet Theory and Thinking about Human Behaviour
4,0.602746,Evolution and diffusion of information literacy topics
5,0.597031,A realist evaluation of a collaborative model to support research co-production in long-term care settings in England: the ExCHANGE protocol
6,0.592725,On exploiting Data Visualization and IoT for Increasing Sustainability and Safety in a Smart Campus
7,0.589483,Academic Literacy Development: What Does It Entail for Multilingual Scholars?
8,0.587543,"Fostering Cognitive Presence, Social Presence and Teaching Presence with Integrated Online—Team-Based Learning"
9,0.578930,Technology and Its Impact on Work–Life Integration



 Query: What has been published about data standards and nomenclature?


,score,title
0,0.613735,On the use of telemedicine in the context of COVID-19: legal aspects and a systematic review of technology
1,0.587784,Das IAB Transparency & Consent Framework – eine Branche rettet sich selbst
2,0.580000,Facet Theory and Thinking about Human Behaviour
3,0.576449,Nominal Equational Problems
4,0.564490,Time series extrinsic regression: Predicting numeric values from time series data
5,0.557044,One-way Resynchronizability of Word Transducers
6,0.550930,Learning Pomset Automata
7,0.548894,Chapter 4 Diagnosis of COVID-19
8,0.547711,"A General Semantic Construction of Dependent Refinement Type Systems, Categorically"
9,0.547628,Evolution and diffusion of information literacy topics



 Query: What has been published about governmental public health?


...

,score,title
0,0.670191,"Conflict, Development, and the Environment in Asia"
1,0.659187,Compressibility of Network Opinion and Spread States in the Laplacian-Eigenvector Basis
2,0.652866,Risk Compensation and COVID-19 Vaccines
3,0.648624,Coping among public safety personnel: A systematic review and meta–analysis
4,0.637681,A Prediction Model of Risk Factors for Complications among SARS-CoV2 Positive Patients: Cases from Jordan
5,0.633071,Autistic traits and negative emotions in the general population during COVID-19: mediating roles of the behavioural immune system and COVID-19 risk perception
6,0.629862,Dynamics of COVID-19 transmission with comorbidity: a data driven modelling based approach
7,0.626396,Network-based Modeling of COVID-19 Dynamics: Early Pandemic Spread in India
8,0.625998,Finding Meaning Amidst COVID-19: An Existential Positive Psychology Model of Suffering
9,0.624478,Dose-response function approach for detecting spreading processes in temporal network data: Exploring social contagion in the Copenhagen Networks Study



 Query: What has been published to clarify community measures?


,score,title
0,0.620999,"Comment on Alley, S.J., et al. As the Pandemic Progresses, How Does Willingness to Vaccinate against COVID-19 Evolve? Int. J. Environ. Res. Public Health 2021, 18, 797"
1,0.616932,Cost-Effectiveness Analysis in Implementation Science: a Research Agenda and Call for Wider Application
2,0.609121,"Working at a remove: continuous, collective, and configurative approaches to qualitative secondary analysis"
3,0.601387,Facet Theory and Thinking about Human Behaviour
4,0.598787,Co-creating physical activity interventions: a mixed methods evaluation approach
5,0.598192,Constructing a universe for the setoid model
6,0.591461,Strength in Diversity: Integrating Community in Primary Health Care to Advance Universal Health Coverage
7,0.586487,Cognitive Focus
8,0.582452,A realist evaluation of a collaborative model to support research co-production in long-term care settings in England: the ExCHANGE protocol
9,0.580023,From Traditional to Outcome-based Public-Private Partnerships: Social Impact Bonds



 Query: What has been published about equity considerations and problems of inequity?


,score,title
0,0.675344,"Accuracy of Households’ Dwelling Valuations, Housing Demand and Mortgage Decisions: Israeli Case"
1,0.674270,Generalized Bounded Linear Logic and its Categorical Semantics
2,0.671536,Cognitive Focus
3,0.664838,Being implicated: on the fittingness of guilt and indignation over outcomes
4,0.652538,Models and numbers: Representing the world or imposing order?
5,0.648725,Nominal Equational Problems
6,0.645338,Lives that Matter: Criminology and Global Security Inequality
7,0.643547,Densities of Almost Surely Terminating Probabilistic Programs are Differentiable Almost Everywhere
8,0.637967,Effects of Causes and Causes of Effects
9,0.637916,Are we speaking the same language? Call for action to improve theory application and reporting in behaviour change research
